In [48]:
from   bs4 import BeautifulSoup
import pandas as pd
import selenium

import splinter
from   splinter import Browser

# webdriver-manager is being phased out for Selenium >=4.6 (see docs)
import webdriver_manager

import datetime
from   importlib.metadata import version
import os
import platform as p
import sys

pad = 20
print(  f"\n{'Executed' : <{pad}} : {datetime.datetime.now().astimezone().strftime('%Y-%m-%d %H:%M:%S %z %Z')}"
        f"\n{'Platform' : <{pad}} : {p.platform(aliased = False, terse = False)}"
        f"\n{'Conda'    : <{pad}} : {os.environ['CONDA_DEFAULT_ENV'] or sys.executable.split('/')[-3]}"
        f"\n{'Python'   : <{pad}} : {p.python_implementation()} {p.python_version()} {sys.executable}")
print(*[f'{name : <{pad}} : {version(name)}'
        for name in ['BeautifulSoup4', 'Pandas', 'Selenium', 'Splinter', 'webdriver-manager']], sep = '\n')


Executed             : 2023-08-31 22:41:38 -0400 EDT
Platform             : macOS-13.5.1-arm64-arm-64bit
Conda                : webscraping
Python               : CPython 3.10.12 /Users/df/anaconda3/envs/webscraping/bin/python
BeautifulSoup4       : 4.12.2
Pandas               : 2.0.3
Selenium             : 4.12.0
Splinter             : 0.19.0
webdriver-manager    : 3.8.6


In [2]:
browser = Browser(driver_name = 'chrome',
                  headless    = False,
                  retry_count = 3)
browser

In [9]:
url = 'https://static.bc-edx.com/data/web/mars_facts/temperature.html'
browser.visit(url = url)

![](img/01.png)

In [10]:
html = browser.html
print(type(html))
print( len(html))

<class 'str'>
472808


In [12]:
soup = BeautifulSoup(markup   =  html,
                     features = 'html.parser')
type(soup)

bs4.BeautifulSoup

In [19]:
rows = soup.find_all(name   = 'tr',
                     class_ = 'data-row')
print(type(rows))
print( len(rows))

<class 'bs4.element.ResultSet'>
1867


In [51]:
print(type(rows[0]))
print(type(rows[0].find_all(name='td')))
print(type(rows[0].find_all(name='td')[0]))
print(type(rows[0].find_all(name='td')[0].text))

rows[0]#.find_all('td')[0].text

<class 'bs4.element.Tag'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.Tag'>
<class 'str'>


<tr class="data-row">
<td>2</td>
<td>2012-08-16</td>
<td>10</td>
<td>155</td>
<td>6</td>
<td>-75.0</td>
<td>739.0</td>
</tr>

In [37]:
list_of_rows = []
for row in rows:                  # for each <tr> Tag in the ResultSet of <tr> Tags...
  td  = row.find_all(name = 'td') # ...get a ResultSet of <td> Tags
  row = [col.text for col in td]  # extract the text of each <td> Tag in the ResultSet of <td> Tags into a list...
  list_of_rows.append(row)        # ...of lists
list_of_rows[:5]

[['2', '2012-08-16', '10', '155', '6', '-75.0', '739.0'],
 ['13', '2012-08-17', '11', '156', '6', '-76.0', '740.0'],
 ['24', '2012-08-18', '12', '156', '6', '-76.0', '741.0'],
 ['35', '2012-08-19', '13', '157', '6', '-74.0', '732.0'],
 ['46', '2012-08-20', '14', '157', '6', '-74.0', '740.0']]

In [50]:
df = pd.DataFrame(data    = list_of_rows,
                  columns = ['id', 'terrestrial_date', 'sol', 'ls', 'month', 'min_temp', 'pressure'])
df

,id,terrestrial_date,sol,ls,month,min_temp,pressure
0,2,2012-08-16,10,155,6,-75.0,739.0
1,13,2012-08-17,11,156,6,-76.0,740.0
2,24,2012-08-18,12,156,6,-76.0,741.0
3,35,2012-08-19,13,157,6,-74.0,732.0
4,46,2012-08-20,14,157,6,-74.0,740.0
...,...,...,...,...,...,...,...
1862,1889,2018-02-23,1973,133,5,-78.0,730.0
1863,1892,2018-02-24,1974,134,5,-77.0,729.0
1864,1894,2018-02-25,1975,134,5,-76.0,729.0
1865,1893,2018-02-26,1976,135,5,-77.0,728.0


In [52]:
df.dtypes

id                  object
terrestrial_date    object
sol                 object
ls                  object
month               object
min_temp            object
pressure            object
dtype: object